In [1]:

import numpy as np
import tensorflow as tf
import pathlib
from pathlib import Path
import librosa
import os


file_list = None
train_ds = None
val_ds = None
test_ds = None
label_names = None
# audio = None
example_audio = None
example_labels = None
example_filenames = None



TRAIN_DIR = None
TEST_DIR = None
DATA_DIR = None
VAL_DIR = None


Divided into directories this way, you can easily load the data using `keras.utils.audio_dataset_from_directory`.

The audio clips are 1 second or less at 16kHz. The `output_sequence_length=16000` pads the short ones to exactly 1 second (and would trim longer ones) so that they can be easily batched.


In [2]:
# def build():
#     global file_list
#     global train_ds
#     global val_ds
#     global label_names
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
#     # file_list = sorted(tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False))

#     seconds=20
#     train_ds, val_ds = tf.keras.utils.audio_dataset_from_directory(
#         directory=TRAIN_DIR,
#         batch_size=64,
#         validation_split=0.2,
#         seed=0,
#         output_sequence_length=16000*seconds,
#         subset='both'
#         ) 
  
#     label_names = np.array(train_ds.class_names)
#     # set_label_names(np.array(train_ds.class_names))
#     print()
#     print("label names:", label_names)
    
#     train_ds.element_spec

In [3]:
def buildTestTrainDS(DIR):
    # Set absolute directory and parameters
    # TRAIN_DIR = os.path.abspath("data/small_train_ds")
    labels = ['orig-16-44-mono', 'upscale-from-mp3-128']

    audio_files = []
    audio_labels = []

    seconds = 40
    sample_rate = 16000
    output_length = sample_rate * seconds

    # Durchsuche den Ordner und lade die Dateien
    for label in labels:
        label_dir = os.path.join(DIR, label)
        
        # Prüfen, ob der Ordner existiert und darauf zugegriffen werden kann
        if os.path.exists(label_dir):
            for filename in os.listdir(label_dir):
                file_path = os.path.join(label_dir, filename)
                
                # Überprüfen, ob der Pfad auch eine Datei ist
                if os.path.isfile(file_path):
                    try:
                        # Lade Audiodatei mit Librosa
                        audio_data, _ = librosa.load(file_path, sr=sample_rate)

                        # Audio auf die gewünschte Länge kürzen oder auffüllen
                        if len(audio_data) > output_length:
                            audio_data = audio_data[:output_length]
                        else:
                            audio_data = tf.pad(audio_data, [[0, output_length - len(audio_data)]])
                        
                        # Füge Audiodatei und Label hinzu
                        audio_files.append(audio_data)
                        audio_labels.append(label)  # Verwende den Ordnernamen als Label
                    except Exception as e:
                        print(f"Fehler beim Laden der Datei {file_path}: {e}")
        else:
            print(f"Ordner {label_dir} existiert nicht oder keine Zugriffsrechte.")
        
    # Konvertiere Listen in TensorFlow-Tensors
    audio_files = tf.convert_to_tensor(audio_files)
    # Wichtig: Füge die Kanal-Dimension hinzu
    audio_files = tf.expand_dims(audio_files, axis=-1)  # Füge eine extra Dimension für den "Kanal" hinzu (Mono)
    # audio_labels = tf.convert_to_tensor(audio_labels)
    audio_labels = tf.convert_to_tensor(audio_labels)

    # Erstelle einen TensorFlow-Dataset aus den Audiodaten und Labels
    dataset = tf.data.Dataset.from_tensor_slices((audio_files, audio_labels))

    # Optional: Batch-Größe und weitere Parameter hinzufügen
    batch_size = 64
    dataset = dataset.batch(batch_size)
    label_names = np.array(labels)

# Ausgabe der Label-Namen
    print("Manuell gesetzte train_ds label names:", label_names)
    return dataset, label_names

In [4]:
def build():
    global file_list
    global train_ds
    global val_ds
    global test_ds
    global label_names
    file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)

    seconds=40
    print(TRAIN_DIR)
    train_ds, label_names = buildTestTrainDS(TRAIN_DIR)
    # train_ds = tf.keras.utils.audio_dataset_from_directory(
    #     directory=TRAIN_DIR,
    #     batch_size=64,
    #     seed=0,
    #     output_sequence_length=16000*seconds,
    #     ) 
    val_ds = tf.keras.utils.audio_dataset_from_directory(
        directory=VAL_DIR,
        batch_size=64,
        seed=0,
        output_sequence_length=16000*seconds,
        ) 
    test_ds = tf.keras.utils.audio_dataset_from_directory(
        directory=TEST_DIR,
        batch_size=64,
        seed=0,
        output_sequence_length=16000*seconds,
        ) 
  
    # label_names = np.array(train_ds.class_names)
    # # set_label_names(np.array(train_ds.class_names))
    # print()
    # print("train_ds label names:", label_names)
    # print("val_ds label names:  ", np.array(val_ds.class_names))
    # print("test_ds label names:  ", np.array(test_ds.class_names))
    
    train_ds.element_spec

In [5]:
# def build():
#     global file_list
#     global train_ds
#     global label_names
    
#     # Lade die Liste der Dateien ohne Shuffle
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
    
#     # Definiere die Label-Namen basierend auf den Ordnernamen
#     label_mapping = {
#         'orig-16-44-mono': 0,
#         'upscale-from-mp3-128': 1
#     }
#     label_names = np.array(list(label_mapping.keys()))
    
#     # Funktion, um das Label aus dem Dateipfad zu extrahieren
#     def extract_label_from_path(file_path):
#         file_path_str = file_path.numpy().decode('utf-8')  # Konvertiere den Tensor zu einem String
#         parts = pathlib.Path(file_path_str).parts  # Teilt den Pfad in seine Teile auf
#         folder_name = parts[-2]  # Nimmt den Ordnernamen (zweitletztes Element im Pfad)
#         return label_mapping[folder_name]

#     # Verwende `map` mit einer Funktion, um den Tensor zu verarbeiten
#     def process_path(file_path):
#         label = tf.py_function(func=extract_label_from_path, inp=[file_path], Tout=tf.int64)
#         return file_path, label

#     # Mappe die Dateipfade zu ihren Labels
#     labeled_files = file_list.map(process_path)

#     # Lade die Audiodateien und verknüpfe sie mit den Labels
#     def load_wav(file_path, label):
#         audio = tf.io.read_file(file_path)
#         audio, _ = tf.audio.decode_wav(audio)
#         return audio, label
    
#     train_ds = labeled_files.map(lambda x, y: load_wav(x, y))
    
#     # Überprüfung der Zuordnung
#     print("Manuell zugewiesene Labels und Dateinamen:")
#     for file_path, label in labeled_files.take(10):
#         print(f"Dateiname: {file_path.numpy().decode('utf-8')}, Label: {label.numpy()}")

In [6]:
# def build():
#     global file_list
#     global train_ds
#     global val_ds
#     global label_names
    
#     # Lade die Liste der Dateien ohne Shuffle
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
    
#     # Definiere die Label-Namen basierend auf den Ordnernamen
#     label_mapping = {
#         'orig-16-44-mono': 0,
#         'upscale-from-mp3-128': 1
#     }
#     label_names = np.array(list(label_mapping.keys()))
    
#     # Funktion, um das Label aus dem Dateipfad zu extrahieren
#     def extract_label_from_path(file_path):
#         file_path_str = file_path.numpy().decode('utf-8')  # Konvertiere den Tensor zu einem String
#         parts = pathlib.Path(file_path_str).parts  # Teilt den Pfad in seine Teile auf
#         folder_name = parts[-2]  # Nimmt den Ordnernamen (zweitletztes Element im Pfad)
#         return label_mapping[folder_name]

#     # Verwende `map` mit einer Funktion, um den Tensor zu verarbeiten
#     def process_path(file_path):
#         label = tf.py_function(func=extract_label_from_path, inp=[file_path], Tout=tf.int64)
#         return file_path, label

#     # Mappe die Dateipfade zu ihren Labels
#     labeled_files = file_list.map(process_path)

#     # Lade die Audiodateien und verknüpfe sie mit den Labels
#     def load_wav(file_path, label):
#         audio = tf.io.read_file(file_path)
#         audio, _ = tf.audio.decode_wav(audio)
#         return audio, label

#     # Erstelle ein Dataset mit den Labels und Audiodaten
#     labeled_dataset = labeled_files.map(lambda x, y: load_wav(x, y))

#     # Aufteilen in Trainings- und Validierungsdatensätze
#     train_size = int(0.8 * len(label_names))  # 80% für Training
#     val_size = len(label_names) - train_size  # 20% für Validierung

#     # Erstelle das Trainings- und Validierungsdataset
#     train_ds = labeled_dataset.take(train_size)
#     val_ds = labeled_dataset.skip(train_size)

#     # Überprüfung der Zuordnung
#     print("Manuell zugewiesene Labels und Dateinamen:")
#     for file_path, label in labeled_files.take(10):
#         print(f"Dateiname: {file_path.numpy().decode('utf-8')}, Label: {label.numpy()}")
    
#     print("Trainingsdatensatz:", train_ds.element_spec)
#     print("Validierungsdatensatz:", val_ds.element_spec)

The dataset now contains batches of audio clips and integer labels. The audio clips have a shape of `(batch, samples, channels)`.


In [7]:
# train_ds.element_spec

This dataset only contains single channel audio, so use the `tf.squeeze` function to drop the extra axis:


In [8]:
def drop():
  global train_ds
  global val_ds
  global test_ds
  
  @tf.autograph.experimental.do_not_convert
  def squeeze(audio, labels):
    audio = tf.squeeze(audio, axis=-1)
    return audio, labels

  train_ds = train_ds.map(squeeze, tf.data.AUTOTUNE)
  val_ds = val_ds.map(squeeze, tf.data.AUTOTUNE)
  test_ds = test_ds.map(squeeze, tf.data.AUTOTUNE)

The `utils.audio_dataset_from_directory` function only returns up to two splits. It's a good idea to keep a test set separate from your validation set.
Ideally you'd keep it in a separate directory, but in this case you can use `Dataset.shard` to split the validation set into two halves. Note that iterating over **any** shard will load **all** the data, and only keep its fraction.


In [9]:
def split():
    global test_ds
    global val_ds
    test_ds = val_ds.shard(num_shards=2, index=0)
    val_ds = val_ds.shard(num_shards=2, index=1)

In [10]:
def extract():
    global example_audio
    global example_labels
    for example_audio, example_labels in train_ds.take(1):  
        print("Audio Shape:", example_audio.shape)
        print("Labels Shape:", example_labels.shape)
        print("Batch Labels:", example_labels.numpy())
        
# def collect_file_paths_and_labels(directory):
#     file_paths = []
#     labels = []
#     for label in os.listdir(directory):
#         label_dir = os.path.join(directory, label)
#         if os.path.isdir(label_dir):
#             for filename in os.listdir(label_dir):
#                 if filename.endswith('.wav'):
#                     file_paths.append(os.path.join(label_dir, filename))
#                     labels.append(1 if label == 'orig-16-44-mono' else 0)  # Mapping der Labels
#     return file_paths, labels   
        
        


# # Funktion zur Extraktion und Zuweisung der Labels
# def extract_and_map():
#     global example_audio
#     global example_labels
#     file_paths, labels = collect_file_paths_and_labels(TRAIN_DIR)
#     for example_audio, example_labels in train_ds.take(1):
#         print("Audio Shape:", example_audio.shape)
#         print("Original Batch Labels:", example_labels.numpy())

#         # Mappen der Labels aus den Dateipfaden
#         batch_labels = []
#         for audio_path in file_paths:
#             if audio_path in example_audio.numpy():
#                 # Holen wir den Index der Datei im Batch
#                 index = list(example_audio.numpy()).index(audio_path)
#                 batch_labels.append(labels[index])
        
#         print("Gemappte Labels aus Verzeichnis:", batch_labels)

In [11]:
# def compare():
#   global example_filenames
#   global example_labels
#   global label_names
#   # Zähler für die tatsächlichen Labels
#   actual_labels_count = 0

#   # Konsolenausgabe zur Überprüfung der Zuordnung
#   for i in range(len(example_filenames)):
#       # Hole das Label aus example_labels
#       label_index = example_labels[i].numpy()  
#       expected_label = label_names[label_index]

#       # Extrahiere das Label aus dem Dateinamen
#       label_in_filename = example_filenames[i].split('_')[0]  # Hier anpassen, falls das Trennzeichen anders ist
      
#       # Zähle die tatsächlichen Labels
#       if label_in_filename == expected_label:
#           actual_labels_count += 1
#           if(label_in_filename =="upscale-16-48"):
#             print('-' * 50)
#             print(f"Index: {i}")
#             print(f"Dateiname: {example_filenames[i]}")
#             print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")
#       else:
#         print('!' * 50)
#         print(f"Index: {i}")
#         print(f"Dateiname: {example_filenames[i]}")
#         print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")

#   # Am Ende die Gesamtanzahl der Labels ausgeben
#   print('-' * 50)  
#   print('-' * 50)
#   print('-' * 50)
#   print(f"Anzahl der tatsächlichen Labels: {actual_labels_count}")
#   print(f"Anzahl der Labels im Dataset: {len(example_labels)}")

In [12]:
# def compare():
#     global example_filenames
#     global label_names
#     global file_list
    
#     actual_labels_count = 0
#     file_list = tf.data.Dataset.list_files(str(TRAIN_DIR / '**/*.wav'), shuffle=False)
#     # Extrahiere Dateinamen aus dem Dataset
#     example_filenames = []
#     for filepath in file_list.take(len(example_audio)):
#         example_filenames.append(pathlib.Path(filepath.numpy().decode('utf-8')).name)

#     #Konsolenausgabe zur Überprüfung der Zuordnung
#     for i in range(len(example_filenames)):
#         # Hole das Label aus example_labels (prüfe, ob .numpy() notwendig ist)
#         label_index = example_labels[i].numpy() if isinstance(example_labels[i], tf.Tensor) else example_labels[i]
#         expected_label = label_names[label_index]

#         # Extrahiere das Label aus dem Dateinamen (Anpassen, falls die Konvention anders ist)
#         label_in_filename = example_filenames[i].split('_')[0]
        
#         # Zähle die tatsächlichen Labels
#         if label_in_filename == expected_label:
#             actual_labels_count += 1
#             if label_in_filename == "upscale-from-mp3-128":
#                 print('-' * 50)
#                 print(f"Index: {i}")
#                 print(f"Dateiname: {example_filenames[i]}")
#                 print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")
#         else:
#             print('!' * 50)
#             print(f"Index: {i}")
#             print(f"Dateiname: {example_filenames[i]}")
#             print(f"Label im Dateinamen: {label_in_filename}, Erwartetes Label: {expected_label}")
#             # Optional: Debugging-Info
#             print(f"Vollstaendiger Dateipfad: {example_filenames[i]}")
    
#     # Endergebnis ausgeben
#     print('-' * 50)
#     print(f"Anzahl der korrekten Labels: {actual_labels_count}")
#     print(f"Anzahl der ueberprueften Labels: {len(example_labels)}")
#     print('-' * 50)
#     for i in range(len(example_filenames)):
#         print(f"Dateiname: {example_filenames[i]}")
#         print(f"Label: {label_names[example_labels[i]]}")

In [13]:
def compare():    
    global example_filenames
    file_labels = []
    example_filenames = []
    for filepath in file_list.take(len(example_audio)):
        example_filenames.append(pathlib.Path(filepath.numpy().decode('utf-8')).name)
    print(f"example filenames: {example_filenames}")
    for dirpath, dirnames, filenames in os.walk(TRAIN_DIR):
        for filename in filenames:
            if filename.endswith('.wav'):
                label = filename.split('_')[0]
                file_labels.append(label)

    # Erstelle ein Mapping für die Labels
    label_mapping = {
        'orig-16-44-mono': 1,  # Beispiel: positiv
        'upscale-from-mp3-128': 0,  # Beispiel: negativ
    }

    # Erstelle eine Liste der gemappten Labels
    mapped_labels = [label_mapping.get(label, -1) for label in file_labels]
    print("Gemappte Labels aus Dateinamen:", mapped_labels)

    # Vergleiche die gemappten Labels mit den Batch-Labels
    for i in range(min(len(mapped_labels), len(example_labels.numpy()))):
        print(f"Index: {i}, Batch Label: {example_labels.numpy()[i]}, Gemapptes Label: {mapped_labels[i]}")

In [14]:
def run(_TRAIN_DIR, _TEST_DIR, _DATA_DIR, _VAL_DIR):
    global TRAIN_DIR, TEST_DIR, DATA_DIR, VAL_DIR, file_list, example_labels, example_audio, train_ds, val_ds, test_ds, label_names, example_audio, example_labels

    TRAIN_DIR =_TRAIN_DIR
    TEST_DIR =_TEST_DIR
    DATA_DIR = _DATA_DIR
    VAL_DIR = _VAL_DIR

    build()
    drop()
    # split()
    extract()
    compare()
    
    return file_list, example_labels, example_audio, train_ds, val_ds, test_ds, label_names, example_audio, example_labels, example_filenames

In [15]:
import numpy as np
import tensorflow as tf
import pathlib
from pathlib import Path
import os
import librosa
def test ():
    global TRAIN_DIR, TEST_DIR, DATA_DIR, VAL_DIR
    TRAIN_DIR = pathlib.Path('data/small_train_ds')
    TEST_DIR = pathlib.Path('data/small_test_ds')
    DATA_DIR = pathlib.Path('data')
    VAL_DIR = pathlib.Path('data/small_val_ds')
    
    build()
    drop()
    split()
    extract()
    # extract_and_map()
    # compare()
    

# test()